In [17]:
#6.1 helper function for the SMO algorithm
#from numpy import *
import numpy as np
def loadDataSet(filename):
    dataMat = []; labelMat = []
    fr = open(filename)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat

In [18]:
def selectJrand(i, m):
    j = i
    while j == i:
        j = int(random.uniform(0, m))
    return j

In [19]:
def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

In [20]:
dataArr, labelArr = loadDataSet('testSet.txt')

In [21]:
dataArr

[[3.542485, 1.977398],
 [3.018896, 2.556416],
 [7.55151, -1.58003],
 [2.114999, -0.004466],
 [8.127113, 1.274372],
 [7.108772, -0.986906],
 [8.610639, 2.046708],
 [2.326297, 0.265213],
 [3.634009, 1.730537],
 [0.341367, -0.894998],
 [3.125951, 0.293251],
 [2.123252, -0.783563],
 [0.887835, -2.797792],
 [7.139979, -2.329896],
 [1.696414, -1.212496],
 [8.117032, 0.623493],
 [8.497162, -0.266649],
 [4.658191, 3.507396],
 [8.197181, 1.545132],
 [1.208047, 0.2131],
 [1.928486, -0.32187],
 [2.175808, -0.014527],
 [7.886608, 0.461755],
 [3.223038, -0.552392],
 [3.628502, 2.190585],
 [7.40786, -0.121961],
 [7.286357, 0.251077],
 [2.301095, -0.533988],
 [-0.232542, -0.54769],
 [3.457096, -0.082216],
 [3.023938, -0.057392],
 [8.015003, 0.885325],
 [8.991748, 0.923154],
 [7.916831, -1.781735],
 [7.616862, -0.217958],
 [2.450939, 0.744967],
 [7.270337, -2.507834],
 [1.749721, -0.961902],
 [1.803111, -0.176349],
 [8.804461, 3.044301],
 [1.231257, -0.568573],
 [2.074915, 1.41055],
 [-0.743036, -1.73

In [22]:
labelArr

[-1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0]

In [23]:
#the simplified SMO algorithm
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    #m * n matrix and 1 * m matrix
    dataMatrix = np.mat(dataMatIn); labelMat = np.mat(classLabels).transpose()
    b = 0; m, n = np.shape(dataMatrix)
    alphas = np.mat(np.zeros((m, 1)))
    iter = 0
    #只有在所有数据集上遍历maxIter次，且不再发生任何alpha修改之后，程序才会停止并退出while循环
    while iter < maxIter:
        #记录alpha是否巳经进行优化
        alphaPairsChanged = 0
        for i in range(m):
            #第i样本的预测的类别
            #https://blog.csdn.net/puqutogether/article/details/44587653
            fXi = float(multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[i,:].T)) + b
            Ei = fXi - float(labelMat[i])
            #一旦alphas等于0或C，那么它们就巳经在“边界”上了，因而不再能够减小或增大，因此也就不值得再对它们进行优化了
            #https://blog.csdn.net/BIT_666/article/details/79891040
            if ((labelMat[i] * Ei < -toler) and (alphas[i] < C)) or ((labelMat[i] * Ei > toler) and (alphas[i] > 0)):
                j = selectJrand(i, m)
                fXj = float(multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[j,:].T)) + b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                #确定边界
                if (labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L == H:
                    print('L == H')
                    continue
                #推导迭代式，eta = K11+K22-2*K12,也是f(x)的二阶导数
                eta = 2.0 * dataMatrix[i,:] * dataMatrix[j,:].T - dataMatrix[i,:] * dataMatrix[i,:].T\
                - dataMatrix[j,:] * dataMatrix[j,:].T
                #如果二阶导数-eta <= 0，说明一阶导数没有最小值，就不做任何改变，本次循环结束直接运行下一次for循环
                if eta >= 0:
                    print('eta >= 0')
                    continue
                #利用公式更新alpha[j]
                alphas[j] -= labelMat[j] * (Ei - Ej) / eta
                alphas[j] = clipAlpha(alphas[j], H, L)
                if (abs(alphas[j] - alphaJold) < 0.00001):
                    print('j not movind enough')
                    continue
                alphas[i] += labelMat[j] * labelMat[i] * (alphaJold - alphas[j])
                b1 = b - Ei - labelMat[i] * (alphas[i] - alphaIold) * dataMatrix[i,:] * dataMatrix[i,:].T\
                - labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[i,:] * dataMatrix[j,:].T
                b2 = b - Ej - labelMat[i] * (alphas[i] - alphaIold) * dataMatrix[i,:] * dataMatrix[j,:].T\
                - labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[j,:] * dataMatrix[j,:].T
                if (0 < alphas[i]) and (C > alphas[i]):
                    b = b1
                elif (0 < alphas[j]) and (C > alphas[j]):
                    b = b2
                else:
                    b = (b1 + b2) / 2.0
                alphaPairsChanged += 1
                print('iter: %d i: %d, pairs changed %d' % (iter, i, alphaPairsChanged))
        if (alphaPairsChanged == 0):
            iter += 1
        else:
            iter = 0
        print('iteration number: %d' % iter)
    return b, alphas

In [24]:
b, alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)

iter: 0 i: 0, pairs changed 1
j not movind enough
j not movind enough
L == H
j not movind enough
L == H
L == H
iter: 0 i: 15, pairs changed 2
L == H
j not movind enough
iter: 0 i: 24, pairs changed 3
j not movind enough
L == H
j not movind enough
j not movind enough
j not movind enough
iter: 0 i: 43, pairs changed 4
j not movind enough
j not movind enough
j not movind enough
L == H
iter: 0 i: 82, pairs changed 5
iter: 0 i: 88, pairs changed 6
iter: 0 i: 94, pairs changed 7
L == H
iteration number: 0
j not movind enough
j not movind enough
L == H
j not movind enough
j not movind enough
j not movind enough
j not movind enough
iter: 0 i: 24, pairs changed 1
j not movind enough
L == H
iter: 0 i: 54, pairs changed 2
j not movind enough
j not movind enough
j not movind enough
j not movind enough
L == H
iter: 0 i: 94, pairs changed 3
L == H
iteration number: 0
j not movind enough
j not movind enough
L == H
j not movind enough
L == H
j not movind enough
j not movind enough
L == H
j not movind 

j not movind enough
iteration number: 5
j not movind enough
L == H
j not movind enough
j not movind enough
j not movind enough
iteration number: 6
j not movind enough
L == H
j not movind enough
j not movind enough
j not movind enough
iteration number: 7
j not movind enough
j not movind enough
j not movind enough
j not movind enough
j not movind enough
iteration number: 8
j not movind enough
j not movind enough
j not movind enough
j not movind enough
j not movind enough
iteration number: 9
j not movind enough
L == H
j not movind enough
j not movind enough
j not movind enough
iteration number: 10
j not movind enough
j not movind enough
j not movind enough
j not movind enough
j not movind enough
iteration number: 11
j not movind enough
j not movind enough
j not movind enough
j not movind enough
j not movind enough
iteration number: 12
j not movind enough
L == H
j not movind enough
j not movind enough
j not movind enough
iteration number: 13
j not movind enough
j not movind enough
j not mo

iteration number: 10
j not movind enough
j not movind enough
j not movind enough
iteration number: 11
j not movind enough
iter: 11 i: 52, pairs changed 1
j not movind enough
iteration number: 0
j not movind enough
j not movind enough
j not movind enough
iteration number: 1
j not movind enough
j not movind enough
j not movind enough
iteration number: 2
j not movind enough
j not movind enough
j not movind enough
iteration number: 3
j not movind enough
j not movind enough
j not movind enough
iteration number: 4
j not movind enough
j not movind enough
j not movind enough
iteration number: 5
j not movind enough
j not movind enough
j not movind enough
iteration number: 6
j not movind enough
j not movind enough
j not movind enough
iteration number: 7
j not movind enough
j not movind enough
j not movind enough
iteration number: 8
j not movind enough
j not movind enough
j not movind enough
iteration number: 9
j not movind enough
j not movind enough
j not movind enough
iteration number: 10
j not

j not movind enough
j not movind enough
iteration number: 2
j not movind enough
j not movind enough
j not movind enough
iteration number: 3
j not movind enough
j not movind enough
j not movind enough
iteration number: 4
j not movind enough
j not movind enough
j not movind enough
iteration number: 5
j not movind enough
j not movind enough
j not movind enough
iteration number: 6
j not movind enough
j not movind enough
j not movind enough
iteration number: 7
j not movind enough
j not movind enough
j not movind enough
iteration number: 8
j not movind enough
j not movind enough
j not movind enough
iteration number: 9
j not movind enough
j not movind enough
j not movind enough
iteration number: 10
j not movind enough
j not movind enough
iter: 10 i: 55, pairs changed 1
iteration number: 0
j not movind enough
j not movind enough
j not movind enough
iteration number: 1
j not movind enough
j not movind enough
j not movind enough
iteration number: 2
j not movind enough
j not movind enough
j not m

iteration number: 34
j not movind enough
iteration number: 35
j not movind enough
iteration number: 36
j not movind enough
iteration number: 37
j not movind enough
iteration number: 38
j not movind enough
iteration number: 39
j not movind enough
iteration number: 40


In [25]:
b

matrix([[-3.78766022]])

In [26]:
alphas[alphas > 0]

matrix([[0.15310292, 0.15405359, 0.05790078, 0.36505729]])

In [27]:
shape(alphas[alphas > 0])

(1, 4)

In [28]:
for i in range(100):
    if alphas[i] > 0.0:
        print(dataArr[i], labelArr[i])

[4.658191, 3.507396] -1.0
[3.457096, -0.082216] -1.0
[2.893743, -1.643468] -1.0
[6.080573, 0.418886] 1.0


In [88]:
#6.4 Speeding up optimization with the full Platt SMO
class optStruct:
    def __init__(self, dataMatIn, classLabels, C, toler):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m, 1)))
        self.b = 0
        self.eCache = mat(zeros((self.m, 2)))
        
def calcEk(oS, k):
    fXk = float(multiply(oS.alphas, oS.labelMat).T * (oS.X * oS.X[k,:].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek

def selectJ(i, oS, Ei):
    """
    内循环启发方式2
    Parameters：
        i - 标号为i的数据的索引值
        oS - 数据结构
        Ei - 标号为i的数据误差
    Returns:
        j, maxK - 标号为j或maxK的数据的索引值
        Ej - 标号为j的数据误差
    """
    maxK, maxDeltaE, Ej = -1, 0, 0
    #根据Ei更新误差缓存
    #Ei保存的两维中，第一维表示Ei的有效性，第二维才是真实值
    oS.eCache[i] = [1, Ei]
    #返回误差不为0的数据的索引值
    #.A转化为array类型，得到想要的非零的向量的行号
    validEcacheList = nonzero(oS.eCache[:, 0].A)[0]
    #如果有不为0的误差
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:
            if k == i:
                continue
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            #找到maxDeltaE
            if (deltaE > maxDeltaE):
                maxK, maxDeltaE, Ej = k, deltaE, Ek
        return maxK, Ej
    #没有不为0的误差
    else:
        #随机选择alpha_j的索引值
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

def updateEk(oS, k):
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1, Ek]

In [89]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    #判断y*f(x)的取值
    if ((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or\
    ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] > 0)):
        j, Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        #y(label)一个为+1，一个为-1
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        #两个label相同
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L == H: print('L == H'); return 0
        eta = 2.0 * oS.X[i,:] * oS.X[j, :].T - oS.X[i,:] * oS.X[i,:].T - oS.X[j,:] * oS.X[j,:].T
        if eta >= 0: print('eta >= 0'); return 0
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        updateEk(oS, j)
        if (abs(oS.alphas[j] - alphaJold) < 0.00001):
            print('j not moving enough')
            return 0
        oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        updateEk(oS, i)
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) *\
        oS.X[i,:] * oS.X[i,:].T - oS.labelMat[j] * \
        (oS.alphas[j] - alphaJold) * oS.X[i,:] * oS.X[j,:].T
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) *\
        oS.X[i,:] * oS.X[j,:].T - oS.labelMat[j] * \
        (oS.alphas[j] - alphaJold) * oS.X[j,:] * oS.X[j,:].T
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2) / 2.0
        return 1
    else:
        return 0

In [90]:
def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup = ('lin', 0)):
    """
    完整的线性SMO算法
    Parameters：
        dataMatIn - 数据矩阵
        classLabels - 数据标签
        C - 松弛变量
        toler - 容错率
        maxIter - 最大迭代次数
        kTup - 包含核函数信息的元组
    Returns:
        oS.b - SMO算法计算的b
        oS.alphas - SMO算法计算的alphas
    """
    #Platt SMO算法是通过一个外循环来选择第一个alpha值的，并且其选择过程会在两种方式之间进行交替：
    #一种方式是在所有数据集上进行单遍扫描，另一种方式则是在非边界alpha中实现单遍扫描。　　
    #所谓非边界alpha指的就是那些不等于边界0或者C的alpha值。对整个数据集的扫描相当容易，而实现非边界alpha值的扫描时，
    #首先需要建立这些alpha值的列表，然后再对这个表进行遍历。同时，该步骤会跳过那些已知的不会改变的alpha值，即。
    #在选择第一个alpha值后，算法会通过一个内循环来选择第二个alpha值。在优化过程中，会通过最大化步长的方式来获得第二个alpha值。
    #在简化版SMO算发放中，我们会在选择j之后计算错误率Ej。
    #但在这里，我们会建立一个全局的缓存用于保存误差值，并从中选择使得步长或者说Ei-Ej最大的alpha值。
    #初始化数据结构
    oS = optStruct(mat(dataMatIn), mat(classLabels).transpose(), C, toler)
    #初始化当前迭代次数
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    #遍历整个数据集alpha都没有更新或者超过最大迭代次数,则退出循环
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        #遍历整个数据集
        if entireSet:
            for i in range(oS.m):
                #使用优化的SMO算法
                alphaPairsChanged += innerL(i, oS)
            print('fullSet, iter: %d i:%d, pairs changed %d' %(iter, i, alphaPairsChanged))
            iter += 1
        #遍历非边界值
        else:
            #遍历不在边界0和C的alpha
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i, oS)
                print('non-bound, iter: %d i:%d, pairs changed %d' % (iter, i, alphaPairsChanged))
            iter +=1
        #遍历一次后改为非边界遍历
        if entireSet: entireSet = False
        elif (alphaPairsChanged == 0): entireSet = True
        print('iteration number: %d' % iter)
    #返回SMO算法计算的b和alphas
    return oS.b, oS.alphas

In [91]:
dataArr, labelArr = loadDataSet('testSet.txt')

In [92]:
b, alphas = smoP(dataArr, labelArr, 0.6, 0.001, 40)

L == H
L == H
L == H
j not moving enough
L == H
L == H
L == H
L == H
L == H
L == H
j not moving enough
L == H
L == H
j not moving enough
L == H
L == H
L == H
L == H
L == H
j not moving enough
fullSet, iter: 0 i:99, pairs changed 6
iteration number: 1
j not moving enough
non-bound, iter: 1 i:0, pairs changed 0
j not moving enough
non-bound, iter: 1 i:3, pairs changed 0
j not moving enough
non-bound, iter: 1 i:4, pairs changed 0
j not moving enough
non-bound, iter: 1 i:17, pairs changed 0
j not moving enough
non-bound, iter: 1 i:18, pairs changed 0
j not moving enough
non-bound, iter: 1 i:25, pairs changed 0
j not moving enough
non-bound, iter: 1 i:46, pairs changed 0
non-bound, iter: 1 i:55, pairs changed 0
non-bound, iter: 1 i:94, pairs changed 0
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L == H
j not moving enough
j not moving enough
L == H
j not moving enough
j not mov

In [93]:
def calcWs(alphas, dataArr, classLabels):
    X = mat(dataArr)
    labelMat = mat(classLabels).transpose()
    m, n = shape(X)
    w = zeros((n, 1))
    for i in range(m):
        w += multiply(alphas[i] * labelMat[i], X[i,:].T)
    return w

In [94]:
ws = calcWs(alphas, dataArr, labelArr)

In [95]:
ws

array([[ 0.65307162],
       [-0.17196128]])

In [96]:
datMat = mat(dataArr)

In [97]:
datMat[0] * mat(ws) + b

matrix([[-0.92555695]])

In [98]:
labelArr[0]

-1.0

In [104]:
def kernelTrans(X, A, kTup):
    m, n = shape(X)
    K = mat(zeros((m, 1)))
    #线性核函数,只进行内积。
    if kTup[0] == 'lin': K = X * A.T
    #高斯核函数,根据高斯核函数公式进行计算
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j,:] - A
            K[j] = deltaRow * deltaRow.T
        K = exp(K / (-1 * kTup[1] ** 2))
    else:
        raise NameError('Houston We Have a Problem -- That Kernel is not recognized')
    return K

class optStruct:
    def __init__(self, dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m, 1)))
        self.b = 0
        self.eCache = mat(zeros((self.m, 2)))
        self.K = mat(zeros((self.m, self.m)))
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup)

In [151]:
def calcEk(oS, k):
    fXk = float(multiply(oS.alphas, oS.labelMat).T * oS.K[:,k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek

#https://blog.csdn.net/hx14301009/article/details/79762782

    
def innerL(i, oS):
    Ei = calcEk(oS, i)
    #判断y*f(x)的取值
    if ((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or\
    ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] > 0)):
        j, Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        #y(label)一个为+1，一个为-1
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        #两个label相同
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L == H: print('L == H'); return 0
        #eta = 2.0 * oS.X[i,:] * oS.X[j, :].T - oS.X[i,:] * oS.X[i,:].T - oS.X[j,:] * oS.X[j,:].T
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j]
        if eta >= 0: print('eta >= 0'); return 0
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        updateEk(oS, j)
        if (abs(oS.alphas[j] - alphaJold) < 0.00001):
            print('j not moving enough')
            return 0
        oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        updateEk(oS, i)
        #b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) *\
        #oS.X[i,:] * oS.X[i,:].T - oS.labelMat[j] * \
        #(oS.alphas[j] - alphaJold) * oS.X[i,:] * oS.X[j,:].T
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) *\
        oS.K[i,i] - oS.labelMat[j] * \
        (oS.alphas[j] - alphaJold) * oS.K[i,j]
        #b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) *\
        #oS.X[i,:] * oS.X[j,:].T - oS.labelMat[j] * \
        #(oS.alphas[j] - alphaJold) * oS.X[j,:] * oS.X[j,:].T
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) *\
        oS.K[i,j] - oS.labelMat[j] * \
        (oS.alphas[j] - alphaJold) * oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2) / 2.0
        return 1
    else:
        return 0

def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup = ('lin', 0)):
    """
    完整的线性SMO算法
    Parameters：
        dataMatIn - 数据矩阵
        classLabels - 数据标签
        C - 松弛变量
        toler - 容错率
        maxIter - 最大迭代次数
        kTup - 包含核函数信息的元组
    Returns:
        oS.b - SMO算法计算的b
        oS.alphas - SMO算法计算的alphas
    """
    #Platt SMO算法是通过一个外循环来选择第一个alpha值的，并且其选择过程会在两种方式之间进行交替：
    #一种方式是在所有数据集上进行单遍扫描，另一种方式则是在非边界alpha中实现单遍扫描。　　
    #所谓非边界alpha指的就是那些不等于边界0或者C的alpha值。对整个数据集的扫描相当容易，而实现非边界alpha值的扫描时，
    #首先需要建立这些alpha值的列表，然后再对这个表进行遍历。同时，该步骤会跳过那些已知的不会改变的alpha值，即。
    #在选择第一个alpha值后，算法会通过一个内循环来选择第二个alpha值。在优化过程中，会通过最大化步长的方式来获得第二个alpha值。
    #在简化版SMO算发放中，我们会在选择j之后计算错误率Ej。
    #但在这里，我们会建立一个全局的缓存用于保存误差值，并从中选择使得步长或者说Ei-Ej最大的alpha值。
    #初始化数据结构
    oS = optStruct(mat(dataMatIn), mat(classLabels).transpose(), C, toler, kTup)
    #初始化当前迭代次数
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    #遍历整个数据集alpha都没有更新或者超过最大迭代次数,则退出循环
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        #遍历整个数据集
        if entireSet:
            for i in range(oS.m):
                #使用优化的SMO算法
                alphaPairsChanged += innerL(i, oS)
            print('fullSet, iter: %d i:%d, pairs changed %d' %(iter, i, alphaPairsChanged))
            iter += 1
        #遍历非边界值
        else:
            #遍历不在边界0和C的alpha
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i, oS)
                print('non-bound, iter: %d i:%d, pairs changed %d' % (iter, i, alphaPairsChanged))
            iter +=1
        #遍历一次后改为非边界遍历
        if entireSet: entireSet = False
        elif (alphaPairsChanged == 0): entireSet = True
        print('iteration number: %d' % iter)
    #返回SMO算法计算的b和alphas
    return oS.b, oS.alphas

In [178]:
def testRbf(k1=1.3):
    dataArr,labelArr = loadDataSet('testSetRBF.txt')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, ('rbf', k1))
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    svInd=nonzero(alphas.A>0)[0]
    sVs=datMat[svInd]
    labelSV = labelMat[svInd];
    print ("there are %d Support Vectors" % shape(sVs)[0])
    m,n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf', k1))
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print ("the training error rate is: %f" % (float(errorCount)/m))
    dataArr,labelArr = loadDataSet('testSetRBF2.txt')
    errorCount = 0
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    m,n = shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf', k1))
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print ("the test error rate is: %f" % (float(errorCount)/m))

In [179]:
testRbf()

L == H
j not moving enough
L == H
j not moving enough
j not moving enough
j not moving enough
L == H
j not moving enough
L == H
L == H
L == H
L == H
L == H
L == H
fullSet, iter: 0 i:99, pairs changed 27
iteration number: 1
j not moving enough
non-bound, iter: 1 i:0, pairs changed 0
j not moving enough
non-bound, iter: 1 i:1, pairs changed 0
j not moving enough
non-bound, iter: 1 i:2, pairs changed 0
j not moving enough
non-bound, iter: 1 i:3, pairs changed 0
j not moving enough
non-bound, iter: 1 i:6, pairs changed 0
j not moving enough
non-bound, iter: 1 i:10, pairs changed 0
j not moving enough
non-bound, iter: 1 i:13, pairs changed 0
j not moving enough
non-bound, iter: 1 i:14, pairs changed 0
non-bound, iter: 1 i:15, pairs changed 1
j not moving enough
non-bound, iter: 1 i:16, pairs changed 1
j not moving enough
non-bound, iter: 1 i:17, pairs changed 1
j not moving enough
non-bound, iter: 1 i:21, pairs changed 1
j not moving enough
non-bound, iter: 1 i:23, pairs changed 1
non-bound

In [202]:
import numpy as np
def img2vector(filename):
    returnVect = zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,i*32+j] = int(lineStr[j])
    return returnVect

def loadImages(dirName):
    from os import listdir
    hwLabels = []
    trainingFileList = listdir(dirName)           #load the training set
    m = len(trainingFileList)
    trainingMat = zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]     #take off .txt
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9: hwLabels.append(-1)
        else: hwLabels.append(1)
        trainingMat[i,:] = img2vector('%s/%s' % (dirName, fileNameStr))
    return trainingMat, hwLabels 

In [205]:
def testDigits(kTup = ('rbf', 10)):
    dataArr, labelArr = loadImages('trainingDigits')
    b, alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, kTup)
    datMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    svInd = nonzero(alphas.A > 0)[0]
    sVs = datMat[svInd]
    labelSV = labelMat[svInd]
    print('there are %d Support Vector' % shape(sVs)[0])
    m, n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i,:], kTup)
        predict = kernelEval.T * multiply(labelSV, alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]): errorCount += 1
    print('the training error rate is: %f' % (float(errorCount) / m))
    dataArr, labelArr = loadImages('testDigits')
    errorCount = 0
    datMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    m, n = shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i,:], kTup)
        predict = kernelEval.T * multiply(labelSV, alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]): errorCount += 1
    print('the test error rate is: %f' % (float(errorCount) / m))

In [206]:
testDigits(('rbf', 20))

j not moving enough
j not moving enough
j not moving enough
L == H
L == H
j not moving enough
L == H
L == H
L == H
j not moving enough
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
j not moving enough
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
j not moving enough
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
j not moving enough
j not moving enough
L == H
j not moving enough
j not moving enough
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
j not moving enou

non-bound, iter: 1 i:258, pairs changed 19
j not moving enough
non-bound, iter: 1 i:260, pairs changed 19
j not moving enough
non-bound, iter: 1 i:277, pairs changed 19
j not moving enough
non-bound, iter: 1 i:286, pairs changed 19
j not moving enough
non-bound, iter: 1 i:290, pairs changed 19
j not moving enough
non-bound, iter: 1 i:296, pairs changed 19
j not moving enough
non-bound, iter: 1 i:308, pairs changed 19
j not moving enough
non-bound, iter: 1 i:332, pairs changed 19
j not moving enough
non-bound, iter: 1 i:339, pairs changed 19
j not moving enough
non-bound, iter: 1 i:346, pairs changed 19
j not moving enough
non-bound, iter: 1 i:354, pairs changed 19
j not moving enough
non-bound, iter: 1 i:361, pairs changed 19
j not moving enough
non-bound, iter: 1 i:362, pairs changed 19
j not moving enough
non-bound, iter: 1 i:364, pairs changed 19
non-bound, iter: 1 i:377, pairs changed 20
j not moving enough
non-bound, iter: 1 i:378, pairs changed 20
j not moving enough
non-bound, i

j not moving enough
non-bound, iter: 2 i:237, pairs changed 2
j not moving enough
non-bound, iter: 2 i:238, pairs changed 2
j not moving enough
non-bound, iter: 2 i:251, pairs changed 2
j not moving enough
non-bound, iter: 2 i:253, pairs changed 2
j not moving enough
non-bound, iter: 2 i:255, pairs changed 2
j not moving enough
non-bound, iter: 2 i:258, pairs changed 2
non-bound, iter: 2 i:260, pairs changed 3
j not moving enough
non-bound, iter: 2 i:277, pairs changed 3
j not moving enough
non-bound, iter: 2 i:286, pairs changed 3
j not moving enough
non-bound, iter: 2 i:288, pairs changed 3
j not moving enough
non-bound, iter: 2 i:290, pairs changed 3
j not moving enough
non-bound, iter: 2 i:296, pairs changed 3
j not moving enough
non-bound, iter: 2 i:308, pairs changed 3
j not moving enough
non-bound, iter: 2 i:332, pairs changed 3
j not moving enough
non-bound, iter: 2 i:339, pairs changed 3
j not moving enough
non-bound, iter: 2 i:346, pairs changed 3
j not moving enough
non-boun

j not moving enough
non-bound, iter: 3 i:51, pairs changed 2
j not moving enough
non-bound, iter: 3 i:55, pairs changed 2
j not moving enough
non-bound, iter: 3 i:58, pairs changed 2
non-bound, iter: 3 i:67, pairs changed 3
j not moving enough
non-bound, iter: 3 i:69, pairs changed 3
j not moving enough
non-bound, iter: 3 i:73, pairs changed 3
j not moving enough
non-bound, iter: 3 i:79, pairs changed 3
j not moving enough
non-bound, iter: 3 i:88, pairs changed 3
j not moving enough
non-bound, iter: 3 i:101, pairs changed 3
j not moving enough
non-bound, iter: 3 i:103, pairs changed 3
j not moving enough
non-bound, iter: 3 i:104, pairs changed 3
j not moving enough
non-bound, iter: 3 i:114, pairs changed 3
j not moving enough
non-bound, iter: 3 i:115, pairs changed 3
j not moving enough
non-bound, iter: 3 i:118, pairs changed 3
j not moving enough
non-bound, iter: 3 i:124, pairs changed 3
j not moving enough
non-bound, iter: 3 i:129, pairs changed 3
j not moving enough
non-bound, iter:

j not moving enough
non-bound, iter: 3 i:1763, pairs changed 4
j not moving enough
non-bound, iter: 3 i:1766, pairs changed 4
j not moving enough
non-bound, iter: 3 i:1772, pairs changed 4
j not moving enough
non-bound, iter: 3 i:1776, pairs changed 4
j not moving enough
non-bound, iter: 3 i:1788, pairs changed 4
j not moving enough
non-bound, iter: 3 i:1793, pairs changed 4
j not moving enough
non-bound, iter: 3 i:1817, pairs changed 4
j not moving enough
non-bound, iter: 3 i:1850, pairs changed 4
j not moving enough
non-bound, iter: 3 i:1864, pairs changed 4
non-bound, iter: 3 i:1865, pairs changed 5
j not moving enough
non-bound, iter: 3 i:1898, pairs changed 5
j not moving enough
non-bound, iter: 3 i:1953, pairs changed 5
iteration number: 4
j not moving enough
non-bound, iter: 4 i:1, pairs changed 0
j not moving enough
non-bound, iter: 4 i:2, pairs changed 0
j not moving enough
non-bound, iter: 4 i:3, pairs changed 0
j not moving enough
non-bound, iter: 4 i:4, pairs changed 0
j no

...

L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L == H
L == H
L == H
j not moving enough
L == H
j not moving enough
L == H
L == H
L == H
L == H
j not moving enough
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
j not moving enough
L == H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L == H
j not moving enough
L == H
L == H
j not moving enough
L == H
j not moving enough
L == H
j not moving enough
L == H
L == H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L == H
L == H
j not moving enough
L == H
L == H
L == H
L == H
L == H
L == H
L == H
j not moving enough
j not moving enough
L == H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L == H
L == H
j not moving enough
j not moving enough
L == H
L == H
L == H
L =

j not moving enough
non-bound, iter: 13 i:290, pairs changed 0
j not moving enough
non-bound, iter: 13 i:296, pairs changed 0
j not moving enough
non-bound, iter: 13 i:308, pairs changed 0
j not moving enough
non-bound, iter: 13 i:332, pairs changed 0
j not moving enough
non-bound, iter: 13 i:339, pairs changed 0
j not moving enough
non-bound, iter: 13 i:346, pairs changed 0
j not moving enough
non-bound, iter: 13 i:354, pairs changed 0
j not moving enough
non-bound, iter: 13 i:361, pairs changed 0
j not moving enough
non-bound, iter: 13 i:362, pairs changed 0
j not moving enough
non-bound, iter: 13 i:364, pairs changed 0
j not moving enough
non-bound, iter: 13 i:377, pairs changed 0
j not moving enough
non-bound, iter: 13 i:378, pairs changed 0
j not moving enough
non-bound, iter: 13 i:379, pairs changed 0
j not moving enough
non-bound, iter: 13 i:380, pairs changed 0
j not moving enough
non-bound, iter: 13 i:388, pairs changed 0
j not moving enough
non-bound, iter: 13 i:394, pairs ch

j not moving enough
non-bound, iter: 13 i:1953, pairs changed 1
iteration number: 14
j not moving enough
non-bound, iter: 14 i:1, pairs changed 0
j not moving enough
non-bound, iter: 14 i:2, pairs changed 0
j not moving enough
non-bound, iter: 14 i:3, pairs changed 0
j not moving enough
non-bound, iter: 14 i:4, pairs changed 0
j not moving enough
non-bound, iter: 14 i:5, pairs changed 0
j not moving enough
non-bound, iter: 14 i:8, pairs changed 0
j not moving enough
non-bound, iter: 14 i:11, pairs changed 0
j not moving enough
non-bound, iter: 14 i:12, pairs changed 0
j not moving enough
non-bound, iter: 14 i:13, pairs changed 0
j not moving enough
non-bound, iter: 14 i:16, pairs changed 0
j not moving enough
non-bound, iter: 14 i:17, pairs changed 0
j not moving enough
non-bound, iter: 14 i:18, pairs changed 0
j not moving enough
non-bound, iter: 14 i:19, pairs changed 0
j not moving enough
non-bound, iter: 14 i:29, pairs changed 0
j not moving enough
non-bound, iter: 14 i:30, pairs c

j not moving enough
non-bound, iter: 14 i:1000, pairs changed 0
j not moving enough
non-bound, iter: 14 i:1047, pairs changed 0
j not moving enough
non-bound, iter: 14 i:1048, pairs changed 0
j not moving enough
non-bound, iter: 14 i:1056, pairs changed 0
j not moving enough
non-bound, iter: 14 i:1057, pairs changed 0
j not moving enough
non-bound, iter: 14 i:1100, pairs changed 0
j not moving enough
non-bound, iter: 14 i:1101, pairs changed 0
j not moving enough
non-bound, iter: 14 i:1108, pairs changed 0
j not moving enough
non-bound, iter: 14 i:1109, pairs changed 0
j not moving enough
non-bound, iter: 14 i:1122, pairs changed 0
j not moving enough
non-bound, iter: 14 i:1184, pairs changed 0
j not moving enough
non-bound, iter: 14 i:1189, pairs changed 0
j not moving enough
non-bound, iter: 14 i:1192, pairs changed 0
j not moving enough
non-bound, iter: 14 i:1196, pairs changed 0
j not moving enough
non-bound, iter: 14 i:1212, pairs changed 0
j not moving enough
non-bound, iter: 14 

L == H
L == H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L == H
L == H
j not moving enough
j not moving enough
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L == H
j not moving enough
j not moving enough
j not moving enough
L == H
L == H
j not moving enough
L == H
L == H
L == H
L == H
j not moving enough
L == H
L == H
j not moving enough
j not moving enough
L == H
L == H
j not moving enough
j not moving enough
j not moving enough
L == H
L == H
j not moving enough
L == H
L == H
L == H
L == H
j not moving enough
L == H
L == H
L == H
L == H
L == H
j not moving enough
j not moving enough
L == H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
j not moving enough
j no

In [ ]:
def testDigits(kTup=('rbf', 10)):
    #获取类别标签及数据
    dataArr,labelArr = loadImages('trainingDigits')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, kTup)
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    svInd=nonzero(alphas.A>0)[0]
    sVs=datMat[svInd] 
    labelSV = labelMat[svInd];
    print ("there are %d Support Vectors" % shape(sVs)[0])
    m,n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print ("the training error rate is: %f" % (float(errorCount)/m))
    dataArr,labelArr = loadImages('testDigits')
    errorCount = 0
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    m,n = shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1    
    print ("the test error rate is: %f" % (float(errorCount)/m)) 